# Chapter 19 - Introduction to Microservice Architecture

## What are microservices?

- An application divided into multiple smaller applications
- Usually deployed as containerized or serverless applications

### Prinicples

- Each microservice should be a cohesive unit of business
- Each microservice should own its data
- Each microservice should be independent of the others

## Cohesive unit of business

- More important to group logical operations together than to try to be "micro"
- Apply the SRP principle
- Easier to break apart than assemble back together

## Ownership of data

- Should share its data through an api
- Should not be shared directly at the database level
- Could share the same database as long as not sharing the same tables
    * Maybe divided up by schema?

## Microservice independence

- Goal is that each service and scale and evolve independenty without needing to change the entire system

## Event-Driven Architecture

- Emitting and consuming events
- Decouples application components or microservices
- Async communication - non-blocking
- Adaptability

### Event types

- Domain events
    * Domain logic
- Integration events
    * Integrate multiple systems together (External systems)
- Application events
    * Internal to a single application
- Enterprise events
    * Cross team

## Message queues

- first in, first out
- `Queue<T>` send messages between components
- `ConcurrentQueue<T>` send messages between threads
- Distributed message queue
    - Azure service bus (As a service)
    - AWS Simple Queue Service
    - Apache ActiveMQ - Open source
    - 
- dead letter queue - stored failed messages

## Publish-Subscribe

- Publisher is unaware of subscribers
- Use message broker i.e. Kafka

## Event sourcing pattern

- store a chronological list of events
- More complex to create and operate
- Eventual consistency
    * Can use http, gRPC to call microservices directly if this is not an option but somes with more drawbacks i.e. performance, less resiliant
        - Death star anti-pattern
    


## Gateway pattern

- Type of facade
- Hide complexity 
    * by routing requests to appropriate services
    * Aggregate responses
    * Expose only a subset of feature
- Ocelot - Open source API gateway
- Reverse proxy 
    * YARP - OS from MS
- Keep resiliency in mind as a gateway can be a single point of failure

### Backend for Frontend pattern (BFF)

- Each front-end has its own specialized gateway
    * i.e. web vs mobile

### Service mesh

- Alternative for microservice communication
- Sidecars





## Project: BFF

- Can use docker to run it so could add sql server, redis cache, event broker, etc

### HTTPS Config

- Must provide a password or it will not work

```pwsh
dotnet dev-certs https -ep "$env:APPDATA\ASP.NET\Https\adpg-net8-chapter-19.pfx" -p devpassword
dotnet dev-certs https --trust
```

These are configured in the `docker-compose.override.yml` file. `ASPNETCORE_Kestrel__Certificates__Default__Path` and `ASPNETCORE_Kestrel__Certificates__Default__Password`

Build the images and start the containers

```bash
cd C19
docker compose build
docker compose up
```

Can use the `.http` files to test the services

### Refit

- Open source library to generate strongly typed http clients for web services

```csharp
using Refit;
using Web.Features;
namespace REPR.BFF;
public interface IBasketsClient
{
    [Get("/baskets/{query.CustomerId}")]
    Task<IEnumerable<Baskets.FetchItems.Item>> FetchCustomerBasketAsync(
        Baskets.FetchItems.Query query,
        CancellationToken cancellationToken);
    [Post("/baskets")]
    Task<Baskets.AddItem.Response> AddProductToCart(
        Baskets.AddItem.Command command,
        CancellationToken cancellationToken);
    [Delete("/baskets/{command.CustomerId}/{command.ProductId}")]
    Task<Baskets.RemoveItem.Response> RemoveProductFromCart(
        Baskets.RemoveItem.Command command,
        CancellationToken cancellationToken);
    [Put("/baskets")]
    Task<Baskets.UpdateQuantity.Response> UpdateProductQuantity(
        Baskets.UpdateQuantity.Command command,
        CancellationToken cancellationToken);
}
```

In [2]:
#r "nuget:Refit"
#r "nuget:Refit.HttpClientFactory"


Installed Packages Refit, 8.0.0 Refit.HttpClientFactory, 8.0.0

In [3]:

using Refit;
using System.Text.Json.Serialization;

public class Breed
{
    public int Id { get; set; }
    public string Name { get; set; }
    [JsonPropertyName("bred_for")]
    public string BredFor { get; set; }
    public string Origin { get; set; }
    [JsonPropertyName("reference_image_id")]
    public string ReferenceImageId { get; set; }
}

public interface IDogApi
{
    [Get("/breeds")]
    Task<List<Breed>> GetBreeds();

    [Get("/breeds/{id}")]
    Task<Breed> GetBreed(int id);
}


Source generation does not seem to work in .net interactive notebooks.

In [4]:

var dogApi = RestService.For<IDogApi>("https://api.thedogapi.com/v1");
var husky = await dogApi.GetBreed(8);
display(husky);

Error: System.InvalidOperationException: IDogApi doesn't look like a Refit interface. Make sure it has at least one method with a Refit HTTP method attribute and Refit is installed in the project.
   at Refit.RestService.GetGeneratedType(Type refitInterfaceType) in c:\temp\releaser\refit\Refit\RestService.cs:line 191
   at System.Collections.Concurrent.ConcurrentDictionary`2.GetOrAdd(TKey key, Func`2 valueFactory)
   at Refit.RestService.For(Type refitInterfaceType, HttpClient client, IRequestBuilder builder) in c:\temp\releaser\refit\Refit\RestService.cs:line 82
   at Refit.RestService.For[T](HttpClient client, IRequestBuilder`1 builder) in c:\temp\releaser\refit\Refit\RestService.cs:line 22
   at Refit.RestService.For[T](HttpClient client, RefitSettings settings) in c:\temp\releaser\refit\Refit\RestService.cs:line 36
   at Refit.RestService.For[T](String hostUrl, RefitSettings settings) in c:\temp\releaser\refit\Refit\RestService.cs:line 56
   at Refit.RestService.For[T](Strin

### Example of aggregation by calling downline services in parallel

```csharp
using System.Collections.Concurrent;
...
app.MapGet(
    "api/cart",
    async (IWebClient client, ICurrentCustomerService currentCustomer, CancellationToken cancellationToken) =>
    {
        var logger = app.Services
            .GetRequiredService<ILoggerFactory>()
            .CreateLogger("GetCart")
        ;
        var basket = await client.Baskets.FetchCustomerBasketAsync(
            new(currentCustomer.Id),
            cancellationToken
        );
        var result = new ConcurrentBag<BasketProduct>();
        await Parallel.ForEachAsync(basket, cancellationToken, async (item, cancellationToken) =>
        {
            logger.LogTrace("Fetching product '{ProductId}'.", item.ProductId);
            var product = await client.Catalog.FetchProductAsync(
                new(item.ProductId),
                cancellationToken
            );
            logger.LogTrace("Found product '{ProductId}' ({ProductName}).", item.ProductId, product.Name);
            result.Add(new BasketProduct(
                product.Id,
                product.Name,
                product.UnitPrice,
                item.Quantity
            ));
        });
        return result;
    }
);
```

## Revisiting the CQRS pattern

- Can go further by dividing them using microservices 
    * Could be serverless
    * i.e
        - Updating the device location (write db)
            * Fire and forget
            * The Saga patter manages transactions across multiple microservices
        - Reading last known location of device (read db)

<svg xmlns="http://www.w3.org/2000/svg" style="background: transparent; background-color: transparent;" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" width="599px" height="199px" viewBox="-0.5 -0.5 599 199" content="&lt;mxfile&gt;&lt;diagram id=&quot;TjMyYshht1W0WdG5TFZE&quot; name=&quot;Page-1&quot;&gt;7Vtbc6M2FP41fgzD/fIY59I+7M5mmna2fcpgUGy6GLEgEnt/fY9AAoTAJjZ24k7imdgcS0I6n853Lsgz42a9+S3z09VXHKJ4pqvhZmbcznRdszUd3qhkW0muTNeuJMssClmrRvAY/UJMqDJpEYUoFxoSjGMSpaIwwEmCAiLI/CzDr2KzZxyLd039JZIEj4Efy9LvUUhWldS11Eb+O4qWK35nTWXfrH3emAnylR/i15bIuJsZNxnGpPq03tygmGqP66Xqdz/wbT2xDCVkTAeGxIsfF2xtbF5kyxe7zHCRzoz5M04IgwJwMOasJ8oI2vRp3V/wEVR5Vlq9VtglCK8RybbQhA1kOqwL2yCgQUexKtFro3DPrUSrlq5dk8HMIF7WgzdagA9MEf1KMd5JKfpupUjrd73TrN/cv37Ytyn9uEIbf4kTWHmKsgjuhLJG+sBFsKr56yoi6DH1A9rtFYgBZCuyjqnmqCKjDeKWrr1FsW0FGjsV6EoKdHRZgfYECrT2K5CuJAJG+eIvUPyA84hEVI23C0wIXou64W2v42hJ2xBMleezqwCWT7U+z0mGf6AbHGOKQYITVOo1jrlophuqOr+7p3BwANebJSVpxf9VZEjxC7i5X01lnuKIjnz3AjfIW7AIw9G/ieE6A0C2BNA1XT+I7oskKNevqzJosBgiQiOB0MVqHYUh7T7PUB79Yuavcu2WS7DmM+uWjgXKz0UL4KoO0bNfxOQ0dnGlCWRry0TjqTIMnMCOgcGRNIxCcLTsku1gYNokRCFTG87ICgO3gOVgagalqv5FhGyZVqgWRYxy4mfkmrr9ZtBSdh/RmZXDoiTkLYLYz/MoqISsyTAj0QkP6p2JclxkARJICG6/RLyV2g9PhmIwxRdx+D51s64PdEc1sHodH6qaHcSqObBeHdDqaYzC0ZXNKSCwcYe8RrGOqwaNvQzyYD/5teDFBYmjBHiPx3oH89HuoOTK6CqUm0DLUoweS7EnsBTvkixFdEQid413Hm+1LJ4cCKZlncS0dOt8tsWX1TKu72gBgus0vZCwbE+wP8Ks7Dr8n9qwNG1/rPYOGZBENv0Z0An1MiIz/KC7bUCz51HbiNyxL66u6wB0maGfr2oaF+h5f4BvqfQF8rjjUKUI9Y0et9fNyjlIJ63Az89RgJQQvcBbrgSgrad0BTN/8pMww1H4lGY4BVcVUUc1dYIxtBU4jTuK4ViCnWluI2ptGE1XnJ4doxsKz/WO2jTmp3ff692tHu9uH+ndRwMkJ/R/ID+c6Xa58kUmgGf/LDBXyFWV0F1DA01NN+Xy+ffwaUnfv+CgSrmnGe4RZdTYLoWtd1qo5r5vcCDXCdogVfo+ADYJswN3kzTO7XwQ9mAL9B2OAnZRMc2XRS3wgx/Lkn++VV6AUwFD2xpJC8NljLYXa1pNsIMMYHTV6WwiTWZ40+hleGeKXXRRZY53onddpnf92ORtNEBy/QIMyl9Ts0gWOX17+Pb4Z1fWi2oZVc1OViPst6Yy1oM9A+bdjQ476FQta3N+g8mNymmcnbZ4pSqm5YjVxisWMB+ZkWv9o/IBIAzN0dGZ+KgizPsWKPssNCiyl5JYaCO0icjfdEzFdtjlP/wW8Pl2077Yti4eWsHCdHZ/iqiu7ArK8retBsyUBjeQpUslHbezX6oh+7s7mthdc906KuFTqNY9RU1Il2tCXXL6Kw19Qh9owGah8/yR4Ff6UCNugs1PMttNZt4+MjMsVxdR1yYhsytTUyyv/WcKt1EVz1bbf94J2K621p1sJ3DL5QYtAHq2/bu+H1yUpKhY/LLhxfKKE+NhpMer+W3WM6cgPZnVbLG8AZKxtCSNJR/K0KWxpit787rZDor7vkKU0Xj6fB3S8PmT1vYdMtmdL0GINg2H6Z7iGmJQpiqaLdCad4ow7d2O7+xWbPdxbH9B3+070jRBDjrmTM9pzjTtP+Th9j0zneCQx5hjOB+zKDbkDc5+kKnnoIykws+TTMOAnQMi53DLlhR/Qv4DM1eEoNUew4eeqaieJmvOsxTTar8mUKRcBfqK8pweBdbVOd2Q2RQl2jpCqf5Du3JbwvsjwZlctr/UM2f6nloQAMvPr/KDZ/2Y7jmHZkzhHb2L9RS7tUytR20HXLLnMBTbcKQWAktNb2tGXwHjIvzKzwIV6GmJEkA6OItr2Y2wpiuAoUilpiLAZToS6pAFQi+5wNCGXbMVwxZeE+D+fy4q8NIqL7O2SgiDldVpqqh9ZVR9oHx1ZF5ndo++6ebhFYXu010IiBTVhEjANh3V4rnQ9MUFQy4uPBSLOMpXMxrYTOLkP8oYQjnkcmslw9HIacslxr5yid49QTSbpHzCn0nWJd9urjBJrcQYUSv5mH54zULzM3jgPXmN1i3smGpPIqO79IGb5GR1rUueB7lVubrT83sW2fdeam6xG5FOWqFrrtaXWU7zmxa4bH6tWVlg86NX4+4/&lt;/diagram&gt;&lt;/mxfile&gt;"><defs/><g><g/><g/><g><path d="M 532 102.5 L 564 102.5 L 584 134.5 L 564 166.5 L 532 166.5 L 512 134.5 Z" fill="#ffffff" stroke="#000000" stroke-miterlimit="10" pointer-events="all" style="fill: light-dark(#ffffff, var(--ge-dark-color, #121212)); stroke: light-dark(rgb(0, 0, 0), rgb(255, 255, 255));"/></g><g><rect x="504" y="102.5" width="72" height="64" fill="none" stroke="none" pointer-events="all"/><path d="M 542.23 132.91 L 551.88 102.5 C 556.13 102.89 560.25 104.19 563.94 106.32 L 551.81 127.58 L 559.19 127.51 L 542.74 145.96 L 548.22 132.91 Z M 545.45 116.41 C 537.43 117.78 531.49 124.5 531.21 132.51 C 530.93 140.52 536.38 147.63 544.27 149.55 C 552.17 151.46 560.35 147.65 563.86 140.43 C 567.38 133.2 565.28 124.53 558.82 119.65 L 563.06 113.53 L 566.13 111.73 L 570.01 115.47 L 566.79 120.73 L 568.32 123.62 L 574.54 123.47 L 576 128.52 L 570.52 131.4 L 570.52 134.79 L 575.93 137.67 L 574.61 142.72 L 568.25 142.57 L 566.72 145.6 L 569.93 150.64 L 566.06 154.46 L 560.8 151.22 L 558.02 152.73 L 558.09 158.93 L 552.97 160.37 L 549.98 154.97 L 546.69 154.97 L 543.69 160.37 L 538.5 158.93 L 538.72 152.81 L 535.8 151.22 L 530.31 154.46 L 526.66 150.64 L 529.95 145.38 L 528.34 142.57 L 522.05 142.72 L 520.81 137.6 L 526.15 134.72 L 526.15 131.47 L 520.74 128.59 L 522.13 123.47 L 528.34 123.62 L 529.95 120.73 L 526.66 115.55 L 530.46 111.87 L 535.94 114.9 L 538.65 113.45 L 538.57 107.18 L 543.62 105.82 L 546.69 111.15 Z M 517.01 143.44 L 518.11 146.75 L 520.08 147.47 L 523.01 145.96 L 524.83 147.76 L 523.37 150.79 L 524.32 152.73 L 527.39 153.74 L 527.54 156.27 L 524.39 157.35 L 523.37 159.15 L 524.98 162.18 L 523.15 163.98 L 520.15 162.68 L 518.11 163.4 L 517.16 166.5 L 514.53 166.5 L 513.43 163.47 L 511.53 162.61 L 508.46 164.19 L 506.56 162.32 L 508.02 159.36 L 507.44 157.56 L 504 156.34 L 504.15 153.82 L 507.22 152.73 L 508.09 150.86 L 506.49 147.91 L 508.31 145.96 L 511.53 147.47 L 513.36 146.75 L 514.31 143.58 L 516.79 143.44 Z M 515.91 151.22 C 514.85 151.22 513.83 151.64 513.07 152.38 C 512.32 153.13 511.89 154.13 511.89 155.18 C 511.89 156.24 512.32 157.24 513.07 157.99 C 513.83 158.73 514.85 159.15 515.91 159.15 C 516.98 159.15 518 158.73 518.76 157.99 C 519.51 157.24 519.94 156.24 519.94 155.18 C 519.94 154.13 519.51 153.13 518.76 152.38 C 518 151.64 516.98 151.22 515.91 151.22" fill="#00bef2" stroke="none" pointer-events="all" style="fill: light-dark(rgb(0, 190, 242), rgb(0, 137, 182));"/></g><g><rect x="503" y="166.5" width="90" height="20" fill="none" stroke="none" pointer-events="all"/></g><g><g transform="translate(-0.5 -0.5)"><switch><foreignObject style="overflow: visible; text-align: left;" pointer-events="none" width="100%" height="100%" requiredFeatures="http://www.w3.org/TR/SVG11/feature#Extensibility"><div xmlns="http://www.w3.org/1999/xhtml" style="display: flex; align-items: unsafe center; justify-content: unsafe center; width: 1px; height: 1px; padding-top: 177px; margin-left: 548px;"><div style="box-sizing: border-box; font-size: 0; text-align: center; color: #000000; "><div style="display: inline-block; font-size: 10px; font-family: &quot;Helvetica&quot;; color: light-dark(#000000, #ffffff); line-height: 1.2; pointer-events: all; white-space: nowrap; ">Azure Function 2</div></div></div></foreignObject><text x="548" y="180" fill="light-dark(#000000, #ffffff)" font-family="&quot;Helvetica&quot;" font-size="10px" text-anchor="middle">Azure Function 2</text></switch></g></g><g><path d="M 30 35 L 63.63 35" fill="none" stroke="#000000" stroke-miterlimit="10" pointer-events="stroke" style="stroke: light-dark(rgb(0, 0, 0), rgb(255, 255, 255));"/><path d="M 68.88 35 L 61.88 38.5 L 63.63 35 L 61.88 31.5 Z" fill="#000000" stroke="#000000" stroke-miterlimit="10" pointer-events="all" style="fill: light-dark(rgb(0, 0, 0), rgb(255, 255, 255)); stroke: light-dark(rgb(0, 0, 0), rgb(255, 255, 255));"/></g><g><ellipse cx="15" cy="12.5" rx="7.5" ry="7.5" fill="#ffffff" stroke="#000000" pointer-events="all" style="fill: light-dark(#ffffff, var(--ge-dark-color, #121212)); stroke: light-dark(rgb(0, 0, 0), rgb(255, 255, 255));"/><path d="M 15 20 L 15 45 M 15 25 L 0 25 M 15 25 L 30 25 M 15 45 L 0 65 M 15 45 L 30 65" fill="none" stroke="#000000" stroke-miterlimit="10" pointer-events="all" style="stroke: light-dark(rgb(0, 0, 0), rgb(255, 255, 255));"/></g><g><g transform="translate(-0.5 -0.5)"><switch><foreignObject style="overflow: visible; text-align: left;" pointer-events="none" width="100%" height="100%" requiredFeatures="http://www.w3.org/TR/SVG11/feature#Extensibility"><div xmlns="http://www.w3.org/1999/xhtml" style="display: flex; align-items: unsafe flex-start; justify-content: unsafe center; width: 1px; height: 1px; padding-top: 72px; margin-left: 15px;"><div style="box-sizing: border-box; font-size: 0; text-align: center; color: #000000; "><div style="display: inline-block; font-size: 10px; font-family: &quot;Helvetica&quot;; color: light-dark(#000000, #ffffff); line-height: 1.2; pointer-events: all; white-space: nowrap; ">Actor</div></div></div></foreignObject><text x="15" y="82" fill="light-dark(#000000, #ffffff)" font-family="&quot;Helvetica&quot;" font-size="10px" text-anchor="middle">Actor</text></switch></g></g><g><path d="M 137.5 35 L 203.63 35" fill="none" stroke="#000000" stroke-miterlimit="10" pointer-events="stroke" style="stroke: light-dark(rgb(0, 0, 0), rgb(255, 255, 255));"/><path d="M 208.88 35 L 201.88 38.5 L 203.63 35 L 201.88 31.5 Z" fill="#000000" stroke="#000000" stroke-miterlimit="10" pointer-events="all" style="fill: light-dark(rgb(0, 0, 0), rgb(255, 255, 255)); stroke: light-dark(rgb(0, 0, 0), rgb(255, 255, 255));"/></g><g><path d="M 90 5 L 117.5 5 L 137.5 35 L 117.5 65 L 90 65 L 70 35 Z" fill="#ffffff" stroke="#000000" stroke-miterlimit="10" pointer-events="all" style="fill: light-dark(#ffffff, var(--ge-dark-color, #121212)); stroke: light-dark(rgb(0, 0, 0), rgb(255, 255, 255));"/></g><g><g transform="translate(-0.5 -0.5)"><switch><foreignObject style="overflow: visible; text-align: left;" pointer-events="none" width="100%" height="100%" requiredFeatures="http://www.w3.org/TR/SVG11/feature#Extensibility"><div xmlns="http://www.w3.org/1999/xhtml" style="display: flex; align-items: unsafe center; justify-content: unsafe center; width: 66px; height: 1px; padding-top: 35px; margin-left: 71px;"><div style="box-sizing: border-box; font-size: 0; text-align: center; color: #000000; "><div style="display: inline-block; font-size: 10px; font-family: &quot;Helvetica&quot;; color: light-dark(#000000, #ffffff); line-height: 1.2; pointer-events: all; white-space: normal; word-wrap: normal; ">Web App</div></div></div></foreignObject><text x="104" y="38" fill="light-dark(#000000, #ffffff)" font-family="&quot;Helvetica&quot;" font-size="10px" text-anchor="middle">Web App</text></switch></g></g><g/><g><path d="M 20 102.5 L 47.5 102.5 L 67.5 132.5 L 47.5 162.5 L 20 162.5 L 0 132.5 Z" fill="#ffffff" stroke="#000000" stroke-miterlimit="10" pointer-events="all" style="fill: light-dark(#ffffff, var(--ge-dark-color, #121212)); stroke: light-dark(rgb(0, 0, 0), rgb(255, 255, 255));"/></g><g><rect x="27.38" y="120.88" width="12.75" height="23.25" fill="none" stroke="none" pointer-events="all"/><path d="M 34.86 123.39 C 35.07 123.39 35.14 123.22 35.14 123.11 C 35.14 123 35.06 122.84 34.86 122.84 L 32.62 122.84 C 32.45 122.84 32.36 122.97 32.36 123.11 C 32.36 123.32 32.52 123.39 32.62 123.39 Z M 35.14 142.23 C 35.49 142.23 35.69 141.98 35.69 141.67 C 35.69 141.41 35.54 141.12 35.14 141.12 L 32.36 141.12 C 32.02 141.12 31.8 141.36 31.8 141.67 C 31.82 141.99 32 142.23 32.36 142.23 Z M 28.48 140.01 L 39.02 140.01 L 39.02 124.5 L 28.48 124.5 Z M 27.38 124.24 C 27.38 123.25 27.96 122.19 29.15 121.75 C 30.89 121.09 35.99 120.88 38.34 121.75 C 39.4 122.16 40.13 123.16 40.13 124.23 L 40.13 140.88 C 40.13 141.72 39.59 142.78 38.5 143.25 C 36.23 144.13 30.77 144.07 28.95 143.23 C 27.94 142.75 27.42 141.85 27.38 140.88 Z" fill="#505050" stroke="none" pointer-events="all" style="fill: light-dark(rgb(80, 80, 80), rgb(168, 168, 168));"/></g><g><path d="M 277.5 35 L 341.7 35" fill="none" stroke="#000000" stroke-miterlimit="10" pointer-events="stroke" style="stroke: light-dark(rgb(0, 0, 0), rgb(255, 255, 255));"/><path d="M 346.95 35 L 339.95 38.5 L 341.7 35 L 339.95 31.5 Z" fill="#000000" stroke="#000000" stroke-miterlimit="10" pointer-events="all" style="fill: light-dark(rgb(0, 0, 0), rgb(255, 255, 255)); stroke: light-dark(rgb(0, 0, 0), rgb(255, 255, 255));"/></g><g><path d="M 230 5 L 257.5 5 L 277.5 35 L 257.5 65 L 230 65 L 210 35 Z" fill="#ffffff" stroke="#000000" stroke-miterlimit="10" pointer-events="all" style="fill: light-dark(#ffffff, var(--ge-dark-color, #121212)); stroke: light-dark(rgb(0, 0, 0), rgb(255, 255, 255));"/></g><g><g transform="translate(-0.5 -0.5)"><switch><foreignObject style="overflow: visible; text-align: left;" pointer-events="none" width="100%" height="100%" requiredFeatures="http://www.w3.org/TR/SVG11/feature#Extensibility"><div xmlns="http://www.w3.org/1999/xhtml" style="display: flex; align-items: unsafe center; justify-content: unsafe center; width: 66px; height: 1px; padding-top: 35px; margin-left: 211px;"><div style="box-sizing: border-box; font-size: 0; text-align: center; color: #000000; "><div style="display: inline-block; font-size: 10px; font-family: &quot;Helvetica&quot;; color: light-dark(#000000, #ffffff); line-height: 1.2; pointer-events: all; white-space: normal; word-wrap: normal; ">Read<br style="font-size: 10px;" />Location<br style="font-size: 10px;" />Service</div></div></div></foreignObject><text x="244" y="38" fill="light-dark(#000000, #ffffff)" font-family="&quot;Helvetica&quot;" font-size="10px" text-anchor="middle">Read...</text></switch></g></g><g><path d="M 348.07 15 C 348.07 6.72 357.86 0 369.94 0 C 375.75 0 381.31 1.58 385.41 4.39 C 389.52 7.21 391.82 11.02 391.82 15 L 391.82 55 C 391.82 63.28 382.03 70 369.94 70 C 357.86 70 348.07 63.28 348.07 55 Z" fill="#ffffff" stroke="#000000" stroke-miterlimit="10" pointer-events="all" style="fill: light-dark(#ffffff, var(--ge-dark-color, #121212)); stroke: light-dark(rgb(0, 0, 0), rgb(255, 255, 255));"/><path d="M 391.82 15 C 391.82 23.28 382.03 30 369.94 30 C 357.86 30 348.07 23.28 348.07 15" fill="none" stroke="#000000" stroke-miterlimit="10" pointer-events="all" style="stroke: light-dark(rgb(0, 0, 0), rgb(255, 255, 255));"/></g><g><g transform="translate(-0.5 -0.5)"><switch><foreignObject style="overflow: visible; text-align: left;" pointer-events="none" width="100%" height="100%" requiredFeatures="http://www.w3.org/TR/SVG11/feature#Extensibility"><div xmlns="http://www.w3.org/1999/xhtml" style="display: flex; align-items: unsafe center; justify-content: unsafe center; width: 42px; height: 1px; padding-top: 35px; margin-left: 349px;"><div style="box-sizing: border-box; font-size: 0; text-align: center; color: #000000; "><div style="display: inline-block; font-size: 10px; font-family: &quot;Helvetica&quot;; color: light-dark(#000000, #ffffff); line-height: 1.2; pointer-events: all; white-space: normal; word-wrap: normal; "><br /><br style="font-size: 10px" />Read<br style="font-size: 10px" />DB</div></div></div></foreignObject><text x="370" y="38" fill="light-dark(#000000, #ffffff)" font-family="&quot;Helvetica&quot;" font-size="10px" text-anchor="middle">Read...</text></switch></g></g><g><path d="M 67.16 133.01 L 121.97 133.85" fill="none" stroke="#000000" stroke-miterlimit="10" pointer-events="stroke" style="stroke: light-dark(rgb(0, 0, 0), rgb(255, 255, 255));"/><path d="M 127.22 133.94 L 120.17 137.33 L 121.97 133.85 L 120.28 130.33 Z" fill="#000000" stroke="#000000" stroke-miterlimit="10" pointer-events="all" style="fill: light-dark(rgb(0, 0, 0), rgb(255, 255, 255)); stroke: light-dark(rgb(0, 0, 0), rgb(255, 255, 255));"/></g><g><g transform="translate(-0.5 -0.5)"><switch><foreignObject style="overflow: visible; text-align: left;" pointer-events="none" width="100%" height="100%" requiredFeatures="http://www.w3.org/TR/SVG11/feature#Extensibility"><div xmlns="http://www.w3.org/1999/xhtml" style="display: flex; align-items: unsafe center; justify-content: unsafe center; width: 1px; height: 1px; padding-top: 134px; margin-left: 96px;"><div style="box-sizing: border-box; font-size: 0; text-align: center; color: #000000; background-color: #ffffff; "><div style="display: inline-block; font-size: 10px; font-family: &quot;Helvetica&quot;; color: light-dark(#000000, #ffffff); line-height: 1.2; pointer-events: all; background-color: light-dark(#ffffff, var(--ge-dark-color, #121212)); white-space: nowrap; "> POST </div></div></div></foreignObject><text x="96" y="137" fill="light-dark(#000000, #ffffff)" font-family="&quot;Helvetica&quot;" font-size="10px" text-anchor="middle"> POST </text></switch></g></g><g><path d="M 552.24 102.5 Q 550 33 398.19 34.69" fill="none" stroke="#000000" stroke-miterlimit="10" pointer-events="stroke" style="stroke: light-dark(rgb(0, 0, 0), rgb(255, 255, 255));"/><path d="M 392.94 34.74 L 399.9 31.17 L 398.19 34.69 L 399.98 38.17 Z" fill="#000000" stroke="#000000" stroke-miterlimit="10" pointer-events="all" style="fill: light-dark(rgb(0, 0, 0), rgb(255, 255, 255)); stroke: light-dark(rgb(0, 0, 0), rgb(255, 255, 255));"/></g><g><g transform="translate(-0.5 -0.5)"><switch><foreignObject style="overflow: visible; text-align: left;" pointer-events="none" width="100%" height="100%" requiredFeatures="http://www.w3.org/TR/SVG11/feature#Extensibility"><div xmlns="http://www.w3.org/1999/xhtml" style="display: flex; align-items: unsafe center; justify-content: unsafe center; width: 1px; height: 1px; padding-top: 45px; margin-left: 506px;"><div style="box-sizing: border-box; font-size: 0; text-align: center; color: #000000; background-color: #ffffff; "><div style="display: inline-block; font-size: 10px; font-family: &quot;Helvetica&quot;; color: light-dark(#000000, #ffffff); line-height: 1.2; pointer-events: all; background-color: light-dark(#ffffff, var(--ge-dark-color, #121212)); white-space: nowrap; "> Update last known location </div></div></div></foreignObject><text x="506" y="48" fill="light-dark(#000000, #ffffff)" font-family="&quot;Helvetica&quot;" font-size="10px" text-anchor="middle"> Update last known location </text></switch></g></g><g><path d="M 387.63 135.06 Q 387.63 135.06 505.63 134.53" fill="none" stroke="#000000" stroke-miterlimit="10" pointer-events="stroke" style="stroke: light-dark(rgb(0, 0, 0), rgb(255, 255, 255));"/><path d="M 510.88 134.51 L 503.9 138.04 L 505.63 134.53 L 503.87 131.04 Z" fill="#000000" stroke="#000000" stroke-miterlimit="10" pointer-events="all" style="fill: light-dark(rgb(0, 0, 0), rgb(255, 255, 255)); stroke: light-dark(rgb(0, 0, 0), rgb(255, 255, 255));"/></g><g><g transform="translate(-0.5 -0.5)"><switch><foreignObject style="overflow: visible; text-align: left;" pointer-events="none" width="100%" height="100%" requiredFeatures="http://www.w3.org/TR/SVG11/feature#Extensibility"><div xmlns="http://www.w3.org/1999/xhtml" style="display: flex; align-items: unsafe center; justify-content: unsafe center; width: 1px; height: 1px; padding-top: 135px; margin-left: 445px;"><div style="box-sizing: border-box; font-size: 0; text-align: center; color: #000000; background-color: #ffffff; "><div style="display: inline-block; font-size: 10px; font-family: &quot;Helvetica&quot;; color: light-dark(#000000, #ffffff); line-height: 1.2; pointer-events: all; background-color: light-dark(#ffffff, var(--ge-dark-color, #121212)); white-space: nowrap; "> When LocationAdded </div></div></div></foreignObject><text x="445" y="138" fill="light-dark(#000000, #ffffff)" font-family="&quot;Helvetica&quot;" font-size="10px" text-anchor="middle"> When LocationAdded </text></switch></g></g><g/><g/><g><path d="M 148 102.5 L 180 102.5 L 200 134.5 L 180 166.5 L 148 166.5 L 128 134.5 Z" fill="#ffffff" stroke="#000000" stroke-miterlimit="10" pointer-events="all" style="fill: light-dark(#ffffff, var(--ge-dark-color, #121212)); stroke: light-dark(rgb(0, 0, 0), rgb(255, 255, 255));"/></g><g><rect x="120" y="102.5" width="72" height="64" fill="none" stroke="none" pointer-events="all"/><path d="M 158.23 132.91 L 167.88 102.5 C 172.13 102.89 176.25 104.19 179.94 106.32 L 167.81 127.58 L 175.19 127.51 L 158.74 145.96 L 164.22 132.91 Z M 161.45 116.41 C 153.43 117.78 147.49 124.5 147.21 132.51 C 146.93 140.52 152.38 147.63 160.27 149.55 C 168.17 151.46 176.35 147.65 179.86 140.43 C 183.38 133.2 181.28 124.53 174.82 119.65 L 179.06 113.53 L 182.13 111.73 L 186.01 115.47 L 182.79 120.73 L 184.32 123.62 L 190.54 123.47 L 192 128.52 L 186.52 131.4 L 186.52 134.79 L 191.93 137.67 L 190.61 142.72 L 184.25 142.57 L 182.72 145.6 L 185.93 150.64 L 182.06 154.46 L 176.8 151.22 L 174.02 152.73 L 174.09 158.93 L 168.97 160.37 L 165.98 154.97 L 162.69 154.97 L 159.69 160.37 L 154.5 158.93 L 154.72 152.81 L 151.8 151.22 L 146.31 154.46 L 142.66 150.64 L 145.95 145.38 L 144.34 142.57 L 138.05 142.72 L 136.81 137.6 L 142.15 134.72 L 142.15 131.47 L 136.74 128.59 L 138.13 123.47 L 144.34 123.62 L 145.95 120.73 L 142.66 115.55 L 146.46 111.87 L 151.94 114.9 L 154.65 113.45 L 154.57 107.18 L 159.62 105.82 L 162.69 111.15 Z M 133.01 143.44 L 134.11 146.75 L 136.08 147.47 L 139.01 145.96 L 140.83 147.76 L 139.37 150.79 L 140.32 152.73 L 143.39 153.74 L 143.54 156.27 L 140.39 157.35 L 139.37 159.15 L 140.98 162.18 L 139.15 163.98 L 136.15 162.68 L 134.11 163.4 L 133.16 166.5 L 130.53 166.5 L 129.43 163.47 L 127.53 162.61 L 124.46 164.19 L 122.56 162.32 L 124.02 159.36 L 123.44 157.56 L 120 156.34 L 120.15 153.82 L 123.22 152.73 L 124.09 150.86 L 122.49 147.91 L 124.31 145.96 L 127.53 147.47 L 129.36 146.75 L 130.31 143.58 L 132.79 143.44 Z M 131.91 151.22 C 130.85 151.22 129.83 151.64 129.07 152.38 C 128.32 153.13 127.89 154.13 127.89 155.18 C 127.89 156.24 128.32 157.24 129.07 157.99 C 129.83 158.73 130.85 159.15 131.91 159.15 C 134.14 159.15 135.94 157.37 135.94 155.18 C 135.94 153 134.14 151.22 131.91 151.22" fill="#00bef2" stroke="none" pointer-events="all" style="fill: light-dark(rgb(0, 190, 242), rgb(0, 137, 182));"/></g><g/><g><rect x="305.91" y="167.95" width="90" height="30" fill="none" stroke="none" pointer-events="all"/></g><g><g transform="translate(-0.5 -0.5)"><switch><foreignObject style="overflow: visible; text-align: left;" pointer-events="none" width="100%" height="100%" requiredFeatures="http://www.w3.org/TR/SVG11/feature#Extensibility"><div xmlns="http://www.w3.org/1999/xhtml" style="display: flex; align-items: unsafe center; justify-content: unsafe center; width: 1px; height: 1px; padding-top: 183px; margin-left: 351px;"><div style="box-sizing: border-box; font-size: 0; text-align: center; color: #000000; "><div style="display: inline-block; font-size: 10px; font-family: &quot;Helvetica&quot;; color: light-dark(#000000, #ffffff); line-height: 1.2; pointer-events: all; white-space: nowrap; ">Message Broker<br style="font-size: 10px" />&amp; Event Store</div></div></div></foreignObject><text x="351" y="186" fill="light-dark(#000000, #ffffff)" font-family="&quot;Helvetica&quot;" font-size="10px" text-anchor="middle">Message Broker...</text></switch></g></g><g><path d="M 334.09 102.5 L 367.73 102.5 L 387.73 135.23 L 367.73 167.95 L 334.09 167.95 L 314.09 135.23 Z" fill="#ffffff" stroke="#000000" stroke-miterlimit="10" pointer-events="all" style="fill: light-dark(#ffffff, var(--ge-dark-color, #121212)); stroke: light-dark(rgb(0, 0, 0), rgb(255, 255, 255));"/></g><g><rect x="322.27" y="127.05" width="57.27" height="16.36" fill="none" stroke="none" pointer-events="all"/><path d="M 322.34 128.26 C 322.55 127.54 323.27 127.05 324.07 127.07 L 374.35 127.07 L 379.55 135.46 L 374.35 143.41 L 323.95 143.41 C 323.47 143.38 323.02 143.17 322.72 142.82 C 322.41 142.47 322.27 142.02 322.34 141.57 Z M 323.49 141.41 C 323.63 141.96 324.16 142.34 324.76 142.33 L 373.83 142.33 L 378.39 135.46 L 373.83 128.1 L 324.76 128.1 C 324.24 128.05 323.73 128.31 323.49 128.75 Z M 325.23 130.91 C 325.27 130.42 325.68 130.01 326.21 129.94 L 339.04 129.94 C 339.57 130.01 339.97 130.42 340.02 130.91 L 340.02 139.78 C 339.88 140.22 339.46 140.52 338.98 140.54 L 326.32 140.54 C 325.84 140.52 325.43 140.22 325.28 139.78 Z M 326.79 132.97 L 326.79 139.24 L 338.58 139.24 L 338.58 132.86 L 333.14 136.7 C 332.76 136.94 332.26 136.94 331.87 136.7 Z M 332.57 135.51 L 338.34 131.29 L 326.84 131.29 Z M 341.64 130.91 C 341.66 130.46 341.99 130.07 342.45 129.94 L 355.45 129.94 C 355.91 130.07 356.23 130.46 356.26 130.91 L 356.26 139.78 C 356.15 140.15 355.84 140.44 355.45 140.54 L 342.45 140.54 C 342.05 140.44 341.75 140.15 341.64 139.78 Z M 343.08 132.91 L 343.08 139.24 L 354.87 139.24 L 354.81 132.91 L 349.5 136.7 C 349.09 136.99 348.52 136.99 348.11 136.7 Z M 348.86 135.51 L 354.7 131.29 L 343.08 131.29 Z M 357.93 130.91 C 357.98 130.42 358.39 130.01 358.92 129.94 L 371.69 129.94 C 372.22 130.01 372.62 130.42 372.67 130.91 L 372.67 139.57 C 372.64 139.86 372.49 140.12 372.25 140.3 C 372.01 140.49 371.71 140.57 371.4 140.54 L 358.92 140.54 C 358.46 140.5 358.07 140.2 357.93 139.78 Z M 359.32 132.91 L 359.32 139.24 L 371.11 139.24 L 371.11 132.91 L 365.68 136.7 C 365.32 136.89 364.88 136.89 364.52 136.7 Z M 365.16 135.51 L 370.94 131.29 L 359.38 131.29 Z" fill="#00bef2" stroke="none" pointer-events="all" style="fill: light-dark(rgb(0, 190, 242), rgb(0, 137, 182));"/></g><g><path d="M 200 134.5 Q 200 134.5 307.83 135.02" fill="none" stroke="#000000" stroke-miterlimit="10" pointer-events="stroke" style="stroke: light-dark(rgb(0, 0, 0), rgb(255, 255, 255));"/><path d="M 313.08 135.04 L 306.06 138.51 L 307.83 135.02 L 306.1 131.51 Z" fill="#000000" stroke="#000000" stroke-miterlimit="10" pointer-events="all" style="fill: light-dark(rgb(0, 0, 0), rgb(255, 255, 255)); stroke: light-dark(rgb(0, 0, 0), rgb(255, 255, 255));"/></g><g><g transform="translate(-0.5 -0.5)"><switch><foreignObject style="overflow: visible; text-align: left;" pointer-events="none" width="100%" height="100%" requiredFeatures="http://www.w3.org/TR/SVG11/feature#Extensibility"><div xmlns="http://www.w3.org/1999/xhtml" style="display: flex; align-items: unsafe center; justify-content: unsafe center; width: 1px; height: 1px; padding-top: 135px; margin-left: 256px;"><div style="box-sizing: border-box; font-size: 0; text-align: center; color: #000000; background-color: #ffffff; border-color: #000000; "><div style="display: inline-block; font-size: 10px; font-family: &quot;Helvetica&quot;; color: light-dark(#000000, #ffffff); line-height: 1.2; pointer-events: all; background-color: light-dark(#ffffff, var(--ge-dark-color, #121212)); border-width: 1px; border-style: solid; border-color: inherit; border: 1px solid light-dark(#000000, #ffffff); white-space: nowrap; ">Publish <br style="font-size: 10px" /><br style="font-size: 10px" /><br style="font-size: 10px" /> LocationAdded </div></div></div></foreignObject><text x="256" y="138" fill="light-dark(#000000, #ffffff)" font-family="&quot;Helvetica&quot;" font-size="10px" text-anchor="middle">Publish...</text></switch></g></g><g><rect x="240" y="125.5" width="28.67" height="21.5" fill="none" stroke="none" pointer-events="all"/><path d="M 240.09 127.02 C 240.02 126.25 240.6 125.57 241.38 125.5 L 267.2 125.5 C 267.97 125.53 268.58 126.15 268.58 126.91 L 268.58 145.45 C 268.67 146.21 268.13 146.89 267.36 147 L 241.31 147 C 240.54 146.89 240 146.21 240.09 145.45 Z M 243.25 128.34 L 254.13 136.64 L 265.04 128.34 Z M 253.13 139.43 L 242.96 131.72 L 242.96 144.19 L 265.73 144.24 L 265.73 131.52 L 255.08 139.49 C 254.5 139.91 253.69 139.89 253.13 139.43 Z" fill-opacity="0.5" fill="#00bef2" stroke="none" pointer-events="all" style="fill: light-dark(rgb(0, 190, 242), rgb(0, 137, 182));"/></g><g><rect x="119" y="166.5" width="90" height="20" fill="none" stroke="none" pointer-events="all"/></g><g><g transform="translate(-0.5 -0.5)"><switch><foreignObject style="overflow: visible; text-align: left;" pointer-events="none" width="100%" height="100%" requiredFeatures="http://www.w3.org/TR/SVG11/feature#Extensibility"><div xmlns="http://www.w3.org/1999/xhtml" style="display: flex; align-items: unsafe center; justify-content: unsafe center; width: 1px; height: 1px; padding-top: 177px; margin-left: 164px;"><div style="box-sizing: border-box; font-size: 0; text-align: center; color: #000000; "><div style="display: inline-block; font-size: 10px; font-family: &quot;Helvetica&quot;; color: light-dark(#000000, #ffffff); line-height: 1.2; pointer-events: all; white-space: nowrap; ">Azure Function 1</div></div></div></foreignObject><text x="164" y="180" fill="light-dark(#000000, #ffffff)" font-family="&quot;Helvetica&quot;" font-size="10px" text-anchor="middle">Azure Function 1</text></switch></g></g></g><switch><g requiredFeatures="http://www.w3.org/TR/SVG11/feature#Extensibility"/><a transform="translate(0,-5)" xlink:href="https://www.drawio.com/doc/faq/svg-export-text-problems" target="_blank"><text text-anchor="middle" font-size="10px" x="50%" y="100%">Text is not SVG - cannot display</text></a></switch></svg>

- SignalR
    * Library for push technology

## Microservice Adapter pattern

- Like the adapter strucutural pattern but applied to Microservices using an event-driven arch

### Possible uses

- Adapting an existing system to another
    * Subscribes to events from the broker then executes logic on the existing system
- Decommissioning a legacy application
    * Migrate depencies away from legacy app to event driven broker system
    * Adapter executes the operations on the legacy app that the depencies previously did
- Adapting an event broker to another
    * temporary or permanent

## Summary

- Microservices come with a cost
- Monoliths still make sense for many applications
    * Migrate to microservices when needed
    * Could leverage pub/sub in a monolith
    * Need to factor in team skill level i.e. DevOps skills are essential for building microservices
    

